# Introduction

In this lab, we will explore PySpark SQL, which is Spark’s high-level API for working with structured data. You will learn how to interact with PySparkSQL, using DataFrame API and SQL query.

Before you can use SQL in Spark, you will first need to create a temporary view. Creating a temporary view is performed by calling the createOrReplaceTempView()function on a DataFrame. Once created, pass any SQL statements in a call to spark.sql():

## Setting Up 

Importing pyspark and pyspark's SQL API

In [1]:
import pyspark
from pyspark.sql import SparkSession

Initializing SparkSession

In [2]:
session = SparkSession.builder.appName('car_app').getOrCreate()
# DOCS 
# https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html?highlight=sparksession#pyspark.sql.SparkSession

Reading the CSV File with Spark DataFrame API

In [3]:
cars = session.read.load('../spark_files/cars.csv', format='csv', header=True, inferSchema=True)  
# Note the read.load w/ (format='csv') kwarg, diff from pandas
print(type(cars))

cars.printSchema()  # schema placement looks kind of like YAML 🔗

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- Car: string (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Model: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- city: string (nullable = true)



## Running SQl Queries

Using the createTempView() method. 

In [10]:
cars.createTempView('temp_cars')  # temp view generated from cars dataframe
session.sql("""SELECT * from temp_cars""").show(10)  # sql query inside the established session
# note that we do not need to assign the temp view to a variable name

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+-------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|quantity|   city|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+-------+
|AMC Ambassador Br...|13.0|        8|       360.0|       175|  3821|        11.0|   73|    US|      25|NewYork|
|  AMC Ambassador DPL|15.0|        8|       390.0|       190|  3850|         8.5|   70|    US|       2|     NJ|
|  AMC Ambassador SST|17.0|        8|       304.0|       150|  3672|        11.5|   72|    US|       4| DALLAS|
|         AMC Concord|19.4|        6|       232.0|        90|  3210|        17.2|   78|    US|      52|  TEXAS|
|         AMC Concord|24.3|        4|       151.0|        90|  3003|        20.1|   80|    US|      42|     OH|
|     AMC Concord d/l|18.1|        6|       258.0|       120|  3410|        15.1|   78|    US|       4|N

Simple Aggrigation: Finding Total Quantity of Cars. 
SUM and GROUP BY

In [12]:
session.sql("""
SELECT car, sum(quantity) AS Total 
FROM temp_cars
GROUP BY car
""").show(10)  
# default show w/o input is 10 anyway
# note the AS total
# note sql is not case sensative so Car == car

+--------------------+-----+
|                 car|Total|
+--------------------+-----+
|Chevrolete Chevel...|  206|
|Volkswagen 1131 D...|   63|
|Chevrolet Monte C...|  582|
|     Ford LTD Landau|  331|
|       Honda Prelude|  400|
|      Chevrolet Nova|  608|
|   Volkswagen Rabbit| 1441|
|     Ford Torino 500|   81|
|        Toyota Camry|  153|
|         Audi 100 LS|   65|
+--------------------+-----+
only showing top 10 rows



Multiple Aggrigations: SUM and MAX

In [15]:
session.sql("""
SELECT 
    Car, 
    sum(quantity) AS Sum,
    max(quantity) AS Max
FROM temp_cars 
GROUP BY Car
ORDER BY Car
""").show()
# 👀 diff cities have different amounts of cars in stock, so 
# the query is showing the sum of the total num of car quantities, 
# and the row with the max quantity 

+--------------------+---+---+
|                 Car|Sum|Max|
+--------------------+---+---+
|AMC Ambassador Br...| 25| 25|
|  AMC Ambassador DPL|  2|  2|
|  AMC Ambassador SST|  4|  4|
|         AMC Concord| 94| 52|
|      AMC Concord DL| 45| 45|
|    AMC Concord DL 6|328|328|
|     AMC Concord d/l|  4|  4|
|         AMC Gremlin|512|214|
|          AMC Hornet|404|172|
|AMC Hornet Sporta...| 90| 90|
|         AMC Matador|833|234|
|    AMC Matador (sw)|302|185|
|           AMC Pacer|221|221|
|       AMC Pacer d/l| 75| 75|
|       AMC Rebel SST|  4|  4|
|  AMC Rebel SST (sw)| 53| 53|
|       AMC Spirit DL|575|575|
|         Audi 100 LS| 65| 65|
|          Audi 100LS|209|164|
|           Audi 4000|362|362|
+--------------------+---+---+
only showing top 20 rows



Count the number of *distinct* cities per car: COUNT and DISTINCT

In [19]:
session.sql("""
SELECT Car, count(DISTINCT city) AS `Number of Cities Located in`
FROM temp_cars
GROUP BY Car
""").show()

+--------------------+---------------------------+
|                 Car|Number of Cities Located in|
+--------------------+---------------------------+
|Chevrolete Chevel...|                          1|
|Volkswagen 1131 D...|                          1|
|Chevrolet Monte C...|                          2|
|     Ford LTD Landau|                          1|
|       Honda Prelude|                          1|
|      Chevrolet Nova|                          3|
|   Volkswagen Rabbit|                          5|
|     Ford Torino 500|                          1|
|        Toyota Camry|                          1|
|         Audi 100 LS|                          1|
|Plymouth Valiant ...|                          1|
|Toyota Corolla Ma...|                          1|
|Oldsmobile Cutlas...|                          1|
|Fiat 124 Sport Coupe|                          1|
|Chevrolet Caprice...|                          3|
|     Volvo 145e (sw)|                          1|
|    Chevrolet Camaro|         

Find the Cars that have quantities greater than 200: WHERE Clause. 

In [20]:
session.sql("""
SELECT * 
FROM temp_cars
WHERE quantity > 200 
ORDER BY quantity DESC;
""").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|quantity|  city|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+------+
|   Plymouth Arrow GS|25.5|        4|       122.0|        96|  2300|        15.5|   77|    US|   75275|    OH|
|Buick Opel Isuzu ...|30.0|        4|       111.0|        80|  2155|        14.8|   77|    US|     814|    OH|
|      Toyota Mark II|19.0|        6|       156.0|       108|  2930|        15.5|   76| Japan|     757| TEXAS|
|Oldsmobile Cutlas...|17.0|        8|       260.0|       110|  4060|        19.0|   77|    US|     752|    NJ|
|       AMC Spirit DL|27.4|        4|       121.0|        80|  2670|        15.0|   79|    US|     575|    OH|
|     Ford Mustang GL|27.0|        4|       140.0|        86|  2790|        15.6|   82|    US|     439|    OH|
|

Find the cars that have a horsepower greater than the average:
SUB-QUERY in the WHERE Clause. 

In [22]:
session.sql("""
SELECT * 
FROM temp_cars
WHERE Horsepower > (SELECT avg(Horsepower) FROM temp_cars);
""").show()
# 'where horsepower greater than average from same table'

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+-------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|quantity|   city|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+-------+
|AMC Ambassador Br...|13.0|        8|       360.0|       175|  3821|        11.0|   73|    US|      25|NewYork|
|  AMC Ambassador DPL|15.0|        8|       390.0|       190|  3850|         8.5|   70|    US|       2|     NJ|
|  AMC Ambassador SST|17.0|        8|       304.0|       150|  3672|        11.5|   72|    US|       4| DALLAS|
|     AMC Concord d/l|18.1|        6|       258.0|       120|  3410|        15.1|   78|    US|       4|NewYork|
|AMC Hornet Sporta...|18.0|        6|       258.0|       110|  2962|        13.5|   71|    US|      90|     NJ|
|         AMC Matador|14.0|        8|       304.0|       150|  3672|        11.5|   73|    US|     131| 

Find the Sum of only Toyota Camrys and Dodge D200s

In [28]:
session.sql("""
SELECT 
    Car, 
    sum(quantity) 
FILTER (WHERE Car IN ('Toyota Camry', 'Datsun 200-SX')) AS `Filtered Results`
FROM temp_cars
GROUP BY Car
""").show(25)
# filter will only allow aggrigate functions to be applied 
# where Filter conditions are satisfied

+--------------------+----------------+
|                 Car|Filtered Results|
+--------------------+----------------+
|Chevrolete Chevel...|            null|
|Volkswagen 1131 D...|            null|
|Chevrolet Monte C...|            null|
|     Ford LTD Landau|            null|
|       Honda Prelude|            null|
|      Chevrolet Nova|            null|
|   Volkswagen Rabbit|            null|
|     Ford Torino 500|            null|
|        Toyota Camry|             153|
|         Audi 100 LS|            null|
|Plymouth Valiant ...|            null|
|Toyota Corolla Ma...|            null|
|Oldsmobile Cutlas...|            null|
|Fiat 124 Sport Coupe|            null|
|Chevrolet Caprice...|            null|
|     Volvo 145e (sw)|            null|
|            Audi Fox|            null|
|    Chevrolet Camaro|            null|
|       Dodge Aspen 6|            null|
|AMC Ambassador Br...|            null|
|    Pontiac Catalina|            null|
|       Ford Maverick|            null|


Regular Expressions

In [32]:
session.sql("""
SELECT 
    Car,
    REGEXP_EXTRACT(Car, ' ([A-Za-z]+)\.') AS `Car title`
FROM temp_cars
""").show(10)
# Extracts anything that comes after a space, 
# followed by a group consisting of 1 or more chars, 
# followed by anything else outside of that 0 or more times.
# Example: 
# https://docs.data.world/documentation/sql/reference/functions/regexp_extract.html



+--------------------+----------+
|                 Car| Car title|
+--------------------+----------+
|AMC Ambassador Br...|Ambassador|
|  AMC Ambassador DPL|Ambassador|
|  AMC Ambassador SST|Ambassador|
|         AMC Concord|    Concor|
|         AMC Concord|    Concor|
|     AMC Concord d/l|   Concord|
|      AMC Concord DL|   Concord|
|    AMC Concord DL 6|   Concord|
|         AMC Gremlin|    Gremli|
|         AMC Gremlin|    Gremli|
+--------------------+----------+
only showing top 10 rows

